<font size="6"> **Donwload SEC 10-K Fillings** </font>

In [1]:
import nltk
import numpy as np
import pandas as pd
import scipy
import pickle
import pprint

from src.nlp_quant import bow_sent
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import re
import os

from tqdm import tqdm

In [ ]:
from src import utils
from src.load_data import load_sec10k, io_utils

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%run ../nb_config.py

In [5]:
cfg = utils.read_conf()

In [6]:
DOC_TYPE = '10-K'
newest_pricing_data = '2018-01-01'

In [7]:
INPATH = os.path.join(io_utils.raw_path, 'sec_10k', '')
INFILE1 = 'metadata.pkl'
INFILE2 = 'sec_10k.pkl'

OUTFILE1 =  'parsed_sentiment_loughran_mcdonald.csv'
os.path.isdir(INPATH)

True

In [8]:
OUTPATH =  os.path.join(io_utils.interim_path, 'sec_10k', '')

# Load SEC 10-K Raw fillings and NLTK Copora

In [26]:
with open(INPATH + INFILE1, 'rb') as file:
    metadata = pickle.load(file)

In [10]:
with open(INPATH + INFILE2, 'rb') as file:
    ten_ks_by_ticker = pickle.load(file)

Load stopwords corpus for removing stopwords and wordnet for lemmatizing.

In [11]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manua\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\manua\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Preprocess SEc 10-Ks

In [13]:
word_pattern = re.compile('\w+')
wlm = WordNetLemmatizer()
lemma_english_stopwords = bow_sent.lemmatize_words(wlm, stopwords.words('english'))

for ticker, ten_ks in ten_ks_by_ticker.items():
    for ten_k in tqdm(ten_ks, desc='Cleaning {} 10-Ks'.format(ticker), unit='10-K'):
        # Clean html
        ten_k['file_clean'] = bow_sent.clean_text(ten_k['file'])
        # word lemmatization
        ten_k['file_lemma'] = bow_sent.lemmatize_words(wlm, word_pattern.findall(ten_k['file_clean']))
        # Remove stopwords
        ten_k['file_lemma'] = [word for word in ten_k['file_lemma'] if word not in lemma_english_stopwords]

Cleaning HON 10-Ks: 100%|██████████| 20/20 [00:48<00:00,  2.42s/10-K]


In [14]:
sentiment_df = bow_sent.get_sentiment_loughran_mcdonald()

In [15]:
# Apply the same preprocessing to these words as the 10-k words
sentiment_df['word'] = bow_sent.lemmatize_words(wlm, sentiment_df['word'].str.lower())
sentiment_df = sentiment_df.drop_duplicates('word')

In [16]:
sentiment_df.sample(5)

,negative,positive,uncertainty,litigious,constraining,interesting,word
936,False,False,False,True,False,False,adjournments
11766,False,False,False,True,False,False,chattel
21582,True,False,False,False,False,False,disruption
51270,True,False,False,False,False,False,outage
11102,True,False,False,False,False,False,cautionary


# Write Preprocessed 10Ks

In [17]:
sentiment_df.to_csv(OUTPATH + OUTFILE1, index=False)

In [27]:
with open(OUTPATH + INFILE1, 'wb') as file:
    pickle.dump(metadata, file)

In [19]:
with open(OUTPATH + INFILE2, 'wb') as file:
    pickle.dump(ten_ks_by_ticker, file)